In [ ]:
import numpy
import pickle

In [ ]:
from snippets.ns_helper import cavity_flow, velocity_term

In [ ]:
def run_cavity():
    nx = 41
    with open('IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dt = .005
    nt = 1000
    
    u, v, p = cavity_flow(u, v, p, nt, dt, dx, 
                         velocity_term, 
                         pressure_poisson, 
                         rtol=1e-4)
    
    return u, v, p

In [ ]:
with open('numpy_ans.pickle', 'rb') as f:
    u, v, p = pickle.load(f)

## Other options for accelerating Python code

## Cython

In [ ]:
%load_ext cython

In [ ]:
%%cython
#%%cython -a

cimport numpy
cimport cython

import numpy

from libc.math cimport sqrt


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
@cython.embedsignature(True)
def pressure_poisson(numpy.ndarray[numpy.float_t, ndim=2] p,
                     numpy.ndarray[numpy.float_t, ndim=2] b,
                     double l2_target):

    cdef numpy.ndarray[numpy.float_t, ndim=2] pn = numpy.zeros_like(p)
    cdef int i, j, n
    cdef double s1, s2, iter_diff
    cdef int I = b.shape[0]
    cdef int J = b.shape[1]

    iter_diff = l2_target + 1

    n = 0
    while iter_diff > l2_target and n <= 500:
        pn = p.copy()
        for i in range(1, I - 1):
            for j in range(1, J - 1):
                p[i, j] = (.25 * (pn[i, j + 1] +
                                  pn[i, j - 1] +
                                  pn[i + 1, j] +
                                  pn[i - 1, j]) -
                                  b[i, j])

        for i in range(I):
            p[i, 0] = p[i, 1]
            p[i, J-1] = 0

        for j in range(J):
            p[0, j] = p[1, j]
            p[I-1, j] = p[I-2, j]

        if n % 10 == 0:
            s1 = 0.0
            s2 = 0.0
            for i in range(I):
                for j in range(J):
                    s1 += (p[i, j] - pn[i, j])**2
                    s2 += pn[i, j]**2
            iter_diff = sqrt(s1 / s2)

        n += 1

    return p

In [ ]:
%timeit run_cavity()

In [ ]:
u_cy, v_cy, p_cy = run_cavity()
assert numpy.allclose(u, u_cy)
assert numpy.allclose(v, v_cy)
assert numpy.allclose(p, p_cy)

## Fortran and `f2py`

In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran
#%%fortran -vvv

SUBROUTINE pressure_poisson(p, b, M, N, l2_target)
IMPLICIT NONE

INTEGER(4), INTENT(IN):: M, N
REAL(8), INTENT(IN):: b(M, N), l2_target
REAL(8), INTENT(INOUT):: p(M, N)
REAL(8):: iter_diff, pn(M, N)
INTEGER(4):: c

!F2PY intent(inout):: b
!F2PY intent(inplace, out):: p
!F2PY real(8), optional, intent(in):: l2_target=1E-4
!F2PY integer(4), intent(hide), depend(p):: m=shape(p, 0), n=shape(p, 1)

    c = 0
    iter_diff = l2_target + 1
    
    DO WHILE(iter_diff > l2_target)
        pn = p
        p(2:M-1, 2:N-1) = .25 * (pn(2:M-1, 3:N) + pn(2:M-1, 1:N-2) + &
                                 pn(3:M, 2:N-1) + pn(1:M-2, 2:N-1)) - b(2:M-1, 2:N-1)
        
        p(1:M, 1) = p(1:M, 2)
        p(1:M, N) = 0
        p(1, 1:N) = p(2, 1:N)
        p(M, 1:N) = p(M - 1, 1:N)
        
        
        IF (MOD(c, 10) .eq. 0) iter_diff = DSQRT(SUM((p - pn)**2)/SUM(pn**2))
        IF (c .eq. 500) EXIT
        
        c = c + 1
    ENDDO
    
END SUBROUTINE pressure_poisson

In [ ]:
%timeit run_cavity()

In [ ]:
u_for, v_for, p_for = run_cavity()
assert numpy.allclose(u, u_for)
assert numpy.allclose(v, v_for)
assert numpy.allclose(p, p_for)